In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations

#Load dữ liệu
df = pd.read_csv('data.csv', header=None)
display(df)

,0,1,2,3,4,5
0,Wine,Chips,Bread,Butter,Milk,Apple
1,Wine,NaN,Bread,Butter,Milk,NaN
2,NaN,NaN,Bread,Butter,Milk,NaN
3,NaN,Chips,NaN,NaN,NaN,Apple
4,Wine,Chips,Bread,Butter,Milk,Apple
5,Wine,Chips,NaN,NaN,Milk,NaN
6,Wine,Chips,Bread,Butter,NaN,Apple
7,Wine,Chips,NaN,NaN,Milk,NaN
8,Wine,NaN,Bread,NaN,NaN,Apple
9,Wine,NaN,Bread,Butter,Milk,NaN


In [2]:
records = []
for i in range(0, df.shape[0]):
    records.append([str(df.values[i, j]) for j in range(0, df.shape[1])])

In [3]:
#chuyển records thành transaction
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
display(df1)

,Apple,Bread,Butter,Chips,Milk,Wine,nan
0,True,True,True,True,True,True,False
1,False,True,True,False,True,True,True
2,False,True,True,False,True,False,True
3,True,False,False,True,False,False,True
4,True,True,True,True,True,True,False
5,False,False,False,True,True,True,True
6,True,True,True,True,False,True,True
7,False,False,False,True,True,True,True
8,True,True,False,False,False,True,True
9,False,True,True,False,True,True,True


In [4]:
frequent_itemsets = apriori(df1, min_support=0.6, use_colnames=True)
display(frequent_itemsets)

,support,itemsets
0,0.681818,(Apple)
1,0.727273,(Bread)
2,0.681818,(Butter)
3,0.636364,(Chips)
4,0.772727,(Milk)
5,0.727273,(Wine)
6,0.818182,(nan)
7,0.636364,"(Wine, Milk)"


In [5]:
# build association rules using support metric
rules = association_rules(frequent_itemsets, metric="support", support_only=True, 
                          min_threshold=0.1)

rules = rules[['antecedents', 'consequents', 'support']]
print(rules)

  antecedents consequents   support
0      (Wine)      (Milk)  0.636364
1      (Milk)      (Wine)  0.636364


# Thuật toán Apriori

In [6]:
# Extract transaction from df
transactions = []
for i in range(len(df)):
    transactions.append([str(df.values[i, j]) for j in range(0, df.shape[1])])
print(transactions)

[['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'], ['Wine', 'nan', 'Bread', 'Butter', 'Milk', 'nan'], ['nan', 'nan', 'Bread', 'Butter', 'Milk', 'nan'], ['nan', 'Chips', 'nan', 'nan', 'nan', 'Apple'], ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'], ['Wine', 'Chips', 'nan', 'nan', 'Milk', 'nan'], ['Wine', 'Chips', 'Bread', 'Butter', 'nan', 'Apple'], ['Wine', 'Chips', 'nan', 'nan', 'Milk', 'nan'], ['Wine', 'nan', 'Bread', 'nan', 'nan', 'Apple'], ['Wine', 'nan', 'Bread', 'Butter', 'Milk', 'nan'], ['nan', 'Chips', 'Bread', 'Butter', 'nan', 'Apple'], ['Wine', 'nan', 'nan', 'Butter', 'Milk', 'Apple'], ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'nan'], ['Wine', 'nan', 'Bread', 'nan', 'Milk', 'Apple'], ['Wine', 'nan', 'Bread', 'Butter', 'Milk', 'Apple'], ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'], ['nan', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'], ['nan', 'Chips', 'nan', 'Butter', 'Milk', 'Apple'], ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'], ['Wine', 'nan'

In [7]:
def frequent_itemsets(D, min_support):
    total_transactions = len(D)

    # Đếm tần suất xuất hiện từng item, bỏ qua giá trị missing
    item_counts = {}
    for transaction in np.array(D):
        for item in transaction:
            if pd.isna(item):  # Bỏ qua giá trị missing
                continue
            item_counts[item] = item_counts.get(item, 0) + 1

    # Tính support (%)
    data = []
    for item, count in item_counts.items():
        support = count / total_transactions
        if support >= min_support:
            data.append([item, count, support])

    # Tạo bảng dạng pandas DataFrame
    df2 = pd.DataFrame(data, columns=["Item", "frequency", "support"])

    # Tính toán tần suất xuất hiện của các cặp item
    pair_counts = {}
    for transaction in np.array(D):
        transaction = [item for item in transaction if pd.notna(item)]  # Bỏ qua giá trị missing
        for pair in combinations(transaction, 2):
            pair_counts[pair] = pair_counts.get(pair, 0) + 1

    # Tính support cho các cặp item
    pair_data = []
    for pair, count in pair_counts.items():
        support = count / total_transactions
        if support >= min_support:
            pair_data.append([pair, count, support])

    # Thêm pair_data vào DataFrame
    pair_df = pd.DataFrame(pair_data, columns=["Item", "frequency", "support"])
    pair_df["Item"] = pair_df["Item"].apply(lambda x: str(x[0]) + ", " + str(x[1]))
    pair_df = pair_df.sort_values(by="Item").reset_index(drop=True)
    df2 = pd.concat([df2, pair_df], ignore_index=True)

    return df2

df2 = display(frequent_itemsets(df, min_support=0.6))

,Item,frequency,support
0,Wine,16,0.727273
1,Chips,14,0.636364
2,Bread,16,0.727273
3,Butter,15,0.681818
4,Milk,17,0.772727
5,Apple,15,0.681818
6,"Wine, Milk",14,0.636364


In [8]:
def association_rules(D, min_support, min_conf):
    total_txns = len(D)

    # Đếm tần suất cho 1-item và 2-itemsets
    pair_freq = {}
    item_freq = {}

    for transaction in np.array(D):
        # Remove nan values and ensure all items are strings
        filtered_transaction = [str(item) for item in transaction if not pd.isna(item)]
        for item in filtered_transaction:
            item_freq[item] = item_freq.get(item, 0) + 1
        for pair in combinations(sorted(filtered_transaction), 2):
            pair = frozenset(pair)
            pair_freq[pair] = pair_freq.get(pair, 0) + 1

    # Lọc 2-itemsets theo min_support
    filtered_pairs = {}
    data = []
    for pair, freq in pair_freq.items():
        support = freq / total_txns
        if support >= min_support:
            filtered_pairs[pair] = freq
            items = ', '.join(pair)
            data.append([items, freq, f"{support}"])

    df_freq = pd.DataFrame(data, columns=['Item', 'Frequency', 'Support'])

    # Tính confidence từ các 2-itemsets thỏa minsup
    rules = []
    for pair, freq_pair in filtered_pairs.items():
        A, B = list(pair)
        # A → B
        conf_AB = freq_pair / item_freq[A]
        if conf_AB >= min_conf:
            rules.append((f"({A})", f"({B})", support, conf_AB))
        # B → A
        conf_BA = freq_pair / item_freq[B]
        if conf_BA >= min_conf:
            rules.append((f"({B})", f"({A})", support, conf_BA))
    df_conf = pd.DataFrame(rules, columns=["Antecedents", "Consequents", "Support", "Confidence"])

    return df_freq, df_conf

association_rules(df, min_support=0.6, min_conf=0.8)

(         Item  Frequency             Support
 0  Wine, Milk         14  0.6363636363636364,
   Antecedents Consequents   Support  Confidence
 0      (Wine)      (Milk)  0.636364    0.875000
 1      (Milk)      (Wine)  0.636364    0.823529)